In [16]:
import numpy as np
import math
import time
import random
import itertools
import queue
import pandas as pd
from IPython.display import display, Markdown

In [17]:
# Dataset number. 1, 2 or 3
dataset = "2"

filename = "data" + dataset + ".txt"
f = open(filename, 'r')
l = f.readline().split()

# number of jobs (13)
n = int(l[0])

# number of machines (5)
m = int(l[1])

# ith job's processing time at jth machine 
cost = []
    
for i in range(n):
    temp = []
    for j in range(m):
        temp.append(0)
    cost.append(temp)
  


for i in range(n):
    line = f.readline().split()
    for j in range(int(len(line)/2)):
        cost[i][j] = int(line[2*j+1])
    
f.close()
cost

[[59, 37, 67, 39, 30],
 [89, 41, 42, 59, 43],
 [18, 56, 75, 95, 75],
 [65, 67, 50, 57, 13],
 [1, 79, 71, 78, 88],
 [49, 100, 30, 76, 36],
 [99, 9, 34, 44, 62],
 [35, 46, 58, 26, 73],
 [8, 98, 97, 20, 73],
 [39, 73, 20, 55, 30],
 [60, 18, 97, 61, 22],
 [71, 1, 4, 88, 52],
 [76, 30, 51, 77, 22]]

In [18]:
def initialization(Npop):
    pop = []
    for i in range(Npop):
        p = list(np.random.permutation(n))
        while p in pop:
            p = list(np.random.permutation(n))
        pop.append(p)
    
    return pop

def calculateObj(sol):
    qTime = queue.PriorityQueue()
    
    qMachines = []
    for i in range(m):
        qMachines.append(queue.Queue())
    
    for i in range(n):
        qMachines[0].put(sol[i])
    
    busyMachines = []
    for i in range(m):
        busyMachines.append(False)
    
    time = 0
    
    job = qMachines[0].get()
    qTime.put((time+cost[job][0], 0, job))
    busyMachines[0] = True
    
    while True:
        time, mach, job = qTime.get()
        if job == sol[n-1] and mach == m-1:
            break
        busyMachines[mach] = False
        if not qMachines[mach].empty():
                j = qMachines[mach].get()
                qTime.put((time+cost[j][mach], mach, j))
                busyMachines[mach] = True
        if mach < m-1:
            if busyMachines[mach+1] == False:
                qTime.put((time+cost[job][mach+1], mach+1, job))
                busyMachines[mach+1] = True
            else:
                qMachines[mach+1].put(job)
            
    return time
        

def selection(pop):
    popObj = []
    for i in range(len(pop)):
        popObj.append([calculateObj(pop[i]), i])

    
    popObj.sort()
    
    distr = []
    distrInd = []
    
    for i in range(len(pop)):
        distrInd.append(popObj[i][1])
        prob = (2*(i+1)) / (len(pop) * (len(pop)+1))
        distr.append(prob)
    
    parents = []
    for i in range(len(pop)):
        parents.append(list(np.random.choice(distrInd, 2, p=distr)))
    
    return parents
    


def crossover(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:2])
    
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    
    child = list(parents[0])
    
    for i in range(pos[0], pos[1]):
        child[i] = -1
    
    p = -1
    for i in range(pos[0], pos[1]):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    
    return child


def mutation(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    
    remJob = sol[pos[1]]
    
    for i in range(pos[1], pos[0], -1):
        sol[i] = sol[i-1]
        
    sol[pos[0]] = remJob
    
    return sol
        
def elitistUpdate(oldPop, newPop):
    bestSolInd = 0
    bestSol = calculateObj(oldPop[0])
    
    for i in range(1, len(oldPop)):
        tempObj = calculateObj(oldPop[i])
        if tempObj < bestSol:
            bestSol = tempObj
            bestSolInd = i
            
    rndInd = random.randint(0,len(newPop)-1)
    
    newPop[rndInd] = oldPop[bestSolInd]
    
    return newPop

# Returns best solution's index number, best solution's objective value and average objective value of the given population.
def findBestSolution(pop):
    bestObj = calculateObj(pop[0])
    avgObj = bestObj
    bestInd = 0
    for i in range(1, len(pop)):
        tObj = calculateObj(pop[i])
        avgObj = avgObj + tObj
        if tObj < bestObj:
            bestObj = tObj
            bestInd = i
            
    return bestInd, bestObj, avgObj/len(pop)

In [37]:
import time
# Number of population
Npop = 3
# Probability of crossover
Pc = 1.0
# Probability of mutation
Pm = 1.0
# Stopping number for generation
stopGeneration = 30000

# Start Timer
t1 = time.process_time()

# Creating the initial population
population = initialization(Npop)

# Run the algorithm for 'stopGeneration' times generation
for i in range(stopGeneration):
    # Selecting parents
    parents = selection(population)
    childs = []
    
    # Apply crossover
    for p in parents:
        r = random.random()
        if r < Pc:
            childs.append(crossover([population[p[0]], population[p[1]]]))
        else:
            if r < 0.5:
                childs.append(population[p[0]])
            else:
                childs.append(population[p[1]])
    
    # Apply mutation 
    for c in childs:
        r = random.random()
        if r < Pm:
            c = mutation(c)

    # Update the population
    population = elitistUpdate(population, childs)
    
    #print(population)
    #print(findBestSolution(population))

# Stop Timer
t2 = time.process_time()
    
# Results Time

bestSol, bestObj, avgObj = findBestSolution(population)
    
print("Population:")
print(population)
print() 

print("Solution:")
print(population[bestSol])
print() 

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" %avgObj)
print()

print("%Gap:")
G = 100 * (bestObj-optimalObjective) / optimalObjective
print("%.2f" %G)
print()

print("CPU Time (s)")
timePassed = (t2-t1)
print("%.2f" %timePassed)

Population:
[[7, 11, 6, 10, 2, 4, 12, 1, 0, 9, 8, 5, 3], [7, 11, 2, 4, 12, 1, 0, 9, 10, 8, 6, 5, 3], [8, 7, 4, 0, 11, 5, 2, 12, 1, 9, 10, 6, 3]]

Solution:
[7, 11, 2, 4, 12, 1, 0, 9, 10, 8, 6, 5, 3]

Objective Value:
929

Average Objective Value of Population:
1010.67

%Gap:
0.98

CPU Time (s)
353.05
